In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json
import os
from scipy.stats import t as student_t

class SyntheticMarketGenerator:
    def __init__(self,
                 global_seed=None,
                 trading_days_per_year=252,
                 years=10,
                 default_bull_drift=0.12,
                 default_bear_drift=-0.10,
                 default_upward_bias=0.08,
                 default_bull_vol=0.15,
                 default_bear_vol=0.25):
        
        if global_seed is not None:
            np.random.seed(global_seed)

        self.trading_days_per_year = trading_days_per_year
        self.years = years
        self.total_days = trading_days_per_year * years

        self.default_bull_drift = default_bull_drift
        self.default_bear_drift = default_bear_drift
        self.default_upward_bias = default_upward_bias
        self.default_bull_vol = default_bull_vol
        self.default_bear_vol = default_bear_vol

        self.params = {
            'flash_crash_prob': 0.0002,
            'flash_crash_magnitude': (-0.15, -0.05),
            'earnings_jump_prob': 0.01,
            'earnings_jump_magnitude': (-0.08, 0.12),
            'degrees_of_freedom': 8,
            'vol_of_vol': 0.05,
            'vol_mean_reversion': 0.80,
            'base_vol': 0.10,
        }

        self.regime_transitions = {
            'bull_to_bull': 0.95,
            'bull_to_bear': 0.01,
            'bull_to_correction': 0.04,
            'bear_to_bear': 0.90,
            'bear_to_bull': 0.10,
            'correction_length': (5, 15),
            'correction_depth': (-0.10, -0.03),
        }

        self.BULL = "bull"
        self.BEAR = "bear"
        self.CORRECTION = "correction"
        self.CRASH = "crash"
        self.RECOVERY = "recovery"

    def generate_stock_data(self, ticker="STK", initial_price=None, randomize_params=True):
        """Generate daily OHLCV data for a single stock and return a DataFrame."""
        if randomize_params:
            bull_drift = np.random.normal(self.default_bull_drift, 0.03)
            bear_drift = np.random.normal(self.default_bear_drift, 0.02)
            upward_bias = np.random.normal(self.default_upward_bias, 0.02)
            bull_vol = np.random.normal(self.default_bull_vol, 0.03)
            bear_vol = np.random.normal(self.default_bear_vol, 0.03)
        else:
            bull_drift = self.default_bull_drift
            bear_drift = self.default_bear_drift
            upward_bias = self.default_upward_bias
            bull_vol = self.default_bull_vol
            bear_vol = self.default_bear_vol

        bull_vol = max(bull_vol, 0.02)
        bear_vol = max(bear_vol, 0.05)

        dates = self._generate_dates_with_offset()
        
        N = len(dates)
        close_prices = np.zeros(N)
        open_prices = np.zeros(N)
        high_prices = np.zeros(N)
        low_prices = np.zeros(N)
        volumes = np.zeros(N)
        regimes = np.array([self.BULL]*N, dtype=object)
        daily_vols = np.zeros(N)
        log_returns = np.zeros(N)

        # Initialize first day properly
        if initial_price is None:
            initial_price = np.random.uniform(50, 150)
            
        # For the first day, we'll generate realistic OHLC values
        current_regime = self.BULL
        daily_vol = bull_vol / np.sqrt(self.trading_days_per_year)
        daily_vols[0] = daily_vol
        
        # Generate first day's price action
        close_prices[0] = initial_price
        
        # First day's open is typically near the previous day's close
        # We'll use a small random deviation from the initial price
        open_deviation = np.random.normal(0, daily_vol)
        open_prices[0] = initial_price * (1 + open_deviation)
        
        # Generate realistic high/low for first day
        if open_deviation > 0:  # If opened up
            high_prices[0] = max(open_prices[0], close_prices[0]) * (1 + abs(np.random.normal(0, daily_vol)))
            low_prices[0] = min(open_prices[0], close_prices[0]) * (1 - abs(np.random.normal(0, daily_vol * 0.5)))
        else:  # If opened down
            high_prices[0] = max(open_prices[0], close_prices[0]) * (1 + abs(np.random.normal(0, daily_vol * 0.5)))
            low_prices[0] = min(open_prices[0], close_prices[0]) * (1 - abs(np.random.normal(0, daily_vol)))
            
        # Ensure OHLC relationships are maintained
        high_prices[0] = max(high_prices[0], open_prices[0], close_prices[0])
        low_prices[0] = min(low_prices[0], open_prices[0], close_prices[0])
        
        # Generate first day's volume
        volumes[0] = self._make_volume(open_deviation, daily_vol, current_regime)

        correction_target = None
        correction_end = None

        # Generate subsequent days
        for i in range(1, N):
            current_regime, correction_target, correction_end = self._update_regime(
                current_regime, i, regimes, correction_target, correction_end
            )
            regimes[i] = current_regime

            drift_annual, vol_annual = self._get_regime_drift_vol(
                current_regime, bull_drift, bear_drift, bull_vol, bear_vol,
                i, correction_target, correction_end
            )

            daily_drift = np.log(1 + drift_annual) / self.trading_days_per_year
            desired_vol = vol_annual / np.sqrt(self.trading_days_per_year)
            daily_drift += upward_bias / self.trading_days_per_year

            daily_vol = (
                self.params['vol_mean_reversion']*desired_vol +
                (1 - self.params['vol_mean_reversion'])*daily_vols[i-1] +
                np.random.normal(0, self.params['vol_of_vol']/self.trading_days_per_year)
            )
            min_daily_vol = self.params['base_vol']/np.sqrt(self.trading_days_per_year)
            daily_vol = max(daily_vol, min_daily_vol)
            daily_vols[i] = daily_vol

            shock = student_t.rvs(df=self.params['degrees_of_freedom'])
            shock /= np.sqrt(self.params['degrees_of_freedom']/(self.params['degrees_of_freedom'] - 2))

            daily_log_return = daily_drift + daily_vol*shock

            if current_regime not in [self.CRASH, self.CORRECTION]:
                daily_log_return = self._special_events(daily_log_return)

            log_returns[i] = daily_log_return
            close_prices[i] = close_prices[i-1]*np.exp(daily_log_return)

            o, h, l = self._make_ohlc(close_prices[i-1], close_prices[i], daily_vol, current_regime)
            open_prices[i], high_prices[i], low_prices[i] = o, h, l

            volumes[i] = self._make_volume(daily_log_return, daily_vol, current_regime)

        # Final validation to ensure OHLC relationships
        for i in range(N):
            high_prices[i] = max(open_prices[i], high_prices[i], low_prices[i], close_prices[i])
            low_prices[i] = min(open_prices[i], high_prices[i], low_prices[i], close_prices[i])

        df = pd.DataFrame({
            'Date': dates,
            'Open': open_prices,
            'High': high_prices,
            'Low': low_prices,
            'Close': close_prices,
            'Volume': volumes.astype(int),
            'Regime': regimes,
            'Volatility': daily_vols,
            'LogReturn': log_returns
        })
        df.set_index('Date', inplace=True)

        df.attrs['ticker'] = ticker
        df.attrs['bull_drift'] = bull_drift
        df.attrs['bear_drift'] = bear_drift
        df.attrs['upward_bias'] = upward_bias
        df.attrs['bull_vol'] = bull_vol
        df.attrs['bear_vol'] = bear_vol

        return df

    # [Rest of the methods remain the same...]
    def _generate_dates_with_offset(self):
        """Generate a list of trading dates with a random offset up to 60 days."""
        offset = np.random.randint(0, 61)
        start_date = datetime.datetime(2010,1,1) + datetime.timedelta(days=offset)

        dates = []
        current = start_date
        while len(dates) < self.total_days:
            if current.weekday() < 5:  # Monday to Friday
                dates.append(current)
            current += datetime.timedelta(days=1)
        return dates

    def _update_regime(self, current_regime, i, regimes, corr_target, corr_end):
        r = np.random.random()
        if current_regime == self.BULL:
            if r < self.regime_transitions['bull_to_bear']:
                current_regime = self.BEAR
            elif r < (self.regime_transitions['bull_to_bear'] +
                      self.regime_transitions['bull_to_correction']):
                current_regime = self.CORRECTION
                dur = np.random.randint(*self.regime_transitions['correction_length'])
                corr_end = i + dur
                corr_target = np.random.uniform(*self.regime_transitions['correction_depth'])
        elif current_regime == self.BEAR:
            if r < self.regime_transitions['bear_to_bull']:
                current_regime = self.RECOVERY
                bear_days = np.sum(regimes[:i] == self.BEAR)
                corr_end = i + int(bear_days*0.5)
        elif current_regime == self.CORRECTION:
            if corr_end is not None and i >= corr_end:
                current_regime = self.BULL
                corr_target = None
                corr_end = None
        elif current_regime == self.RECOVERY:
            if corr_end is not None and i >= corr_end:
                current_regime = self.BULL
                corr_end = None
        elif current_regime == self.CRASH:
            current_regime = self.BULL
        return current_regime, corr_target, corr_end

    def _get_regime_drift_vol(self, regime, bull_drift, bear_drift,
                              bull_vol, bear_vol,
                              day_i, corr_target, corr_end):
        """Return annual drift & vol depending on the current regime."""
        if regime == self.BULL:
            drift = bull_drift
            vol   = bull_vol
        elif regime == self.BEAR:
            drift = bear_drift
            vol   = bear_vol
        elif regime == self.CORRECTION:
            if corr_target is None:
                corr_target = np.random.uniform(*self.regime_transitions['correction_depth'])
            drift = corr_target
            vol   = 0.5*(bull_vol + bear_vol)
        elif regime == self.RECOVERY:
            drift = bull_drift*1.5
            vol   = bull_vol + 0.3*(bear_vol - bull_vol)
        elif regime == self.CRASH:
            drift = np.random.uniform(*self.params['flash_crash_magnitude'])
            vol   = bear_vol*2
        else:
            drift = bull_drift
            vol   = bull_vol
        return drift, vol

    def _special_events(self, daily_log_return):
        """Flash crash or earnings jump with given probabilities."""
        if np.random.random() < self.params['flash_crash_prob']:
            return np.random.uniform(*self.params['flash_crash_magnitude'])
        if np.random.random() < self.params['earnings_jump_prob']:
            if np.random.random() < 0.55:
                jump = np.random.uniform(0, self.params['earnings_jump_magnitude'][1])
            else:
                jump = np.random.uniform(self.params['earnings_jump_magnitude'][0], 0)
            return daily_log_return + jump
        return daily_log_return

    def _make_ohlc(self, prev_close, curr_close, daily_vol, regime):
        """Construct open/high/low from close-to-close movement + random intraday range."""
        if regime in [self.BEAR, self.CRASH]:
            daily_range = 0.03
        else:
            daily_range = 0.02

        bull_daily_vol = self.default_bull_vol / np.sqrt(self.trading_days_per_year)
        factor = daily_vol / bull_daily_vol if bull_daily_vol > 0 else 1
        daily_range *= factor

        open_frac = np.clip(np.random.normal(0.5, 0.2), 0, 1)
        open_price = prev_close + (curr_close - prev_close)*open_frac

        if curr_close > prev_close:
            up_wick = np.random.uniform(0, daily_range*0.7)
            down_wick = np.random.uniform(0, daily_range*0.3)
        else:
            up_wick = np.random.uniform(0, daily_range*0.3)
            down_wick = np.random.uniform(0, daily_range*0.7)

        high_price = max(open_price, curr_close) + up_wick
        low_price  = min(open_price, curr_close) - down_wick

        if high_price < low_price:
            high_price = low_price * 1.001

        return open_price, high_price, low_price

    def _make_volume(self, daily_log_return, daily_vol, regime):
        """Pick daily volume starting ~1M, scaled by daily vol & big moves, etc."""
        base_volume = 1_000_000
        bull_daily_vol = self.default_bull_vol / np.sqrt(self.trading_days_per_year)
        if bull_daily_vol <= 0:
            bull_daily_vol = 1e-9

        vol_factor  = 1 + 1.5*(daily_vol / bull_daily_vol - 1)
        move_factor = 1
        if daily_vol > 0:
            move_factor = 1 + 0.8*(abs(daily_log_return)/daily_vol)
        random_factor = np.random.lognormal(0, 0.6)

        volume = base_volume * vol_factor * move_factor * random_factor

        if regime == self.CRASH:
            volume *= 5
        elif regime == self.BEAR:
            volume *= 1.3

        return volume

    def plot_stock(self, df, ticker, save_path):
        """
        Basic plot: line chart of 'Close' with color shading for each regime.
        Saves the figure to 'save_path'.
        """
        fig, ax = plt.subplots(figsize=(10,6))
        ax.plot(df.index, df['Close'], 'k-', lw=1.5, label='Close')

        regime_colors = {
            self.BULL: 'lightgreen',
            self.BEAR: 'lightcoral',
            self.CORRECTION: 'lightyellow',
            self.CRASH: 'red',
            self.RECOVERY: 'lightblue'
        }

        max_y = df['Close'].max() * 1.1
        for regime_val, color in regime_colors.items():
            mask = (df['Regime'] == regime_val)
            if mask.any():
                ax.fill_between(
                    df.index, 0, max_y,
                    where=mask, color=color, alpha=0.2,
                    label=regime_val
                )

        ax.set_title(f"{ticker} Synthetic Price", fontsize=14)
        ax.set_ylabel("Price")
        ax.grid(True, alpha=0.3)
        ax.legend(loc='best')
        fig.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close(fig)

    def generate_random_ticker(self, length=4):
        """
        Create a random uppercase 'ticker' name of given length, e.g. 'ABCD'.
        """
        letters = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
        # We'll pick random letters from the above
        # If you want to ensure uniqueness, you could store used tickers in a set,
        # but for demonstration, this is fine.
        arr = np.random.choice(letters, size=length, replace=True)
        return "".join(arr)

    def generate_portfolio(self, num_stocks=5, output_dir="synthetic_portfolio"):
        """
        Generate data for 'num_stocks' random tickers,
        save CSV + PNG + JSON for each.
        """
        os.makedirs(output_dir, exist_ok=True)
        all_stocks = {}

        for _ in range(num_stocks):
            # Make a random 3- or 4-letter ticker name
            name_len = np.random.choice([3,4])
            ticker_name = self.generate_random_ticker(length=name_len)

            df = self.generate_stock_data(ticker=ticker_name)
            all_stocks[ticker_name] = df

            # Save CSV
            csv_path = os.path.join(output_dir, f"{ticker_name}.csv")
            df.to_csv(csv_path)

            # Save figure
            fig_path = os.path.join(output_dir, f"{ticker_name}.png")
            self.plot_stock(df, ticker_name, fig_path)

            # Save metadata as JSON
            meta = {
                'ticker': ticker_name,
                'bull_drift': df.attrs['bull_drift'],
                'bear_drift': df.attrs['bear_drift'],
                'upward_bias': df.attrs['upward_bias'],
                'bull_vol': df.attrs['bull_vol'],
                'bear_vol': df.attrs['bear_vol']
            }
            meta_path = os.path.join(output_dir, f"{ticker_name}_metadata.json")
            with open(meta_path, 'w') as f:
                json.dump(meta, f, indent=2)

        print(f"Generated {num_stocks} random stocks into '{output_dir}'.")
        return all_stocks

def main():
    # Example usage: generate 5 random stocks with a global seed
    gen = SyntheticMarketGenerator(global_seed=42)
    gen.generate_portfolio(num_stocks=50, output_dir="random_ticker_portfolio")

if __name__ == "__main__":
    main()


Generated 50 random stocks into 'random_ticker_portfolio'.


# BEAT OR EQUALIZE SNP500

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import json
import os
from scipy.stats import t as student_t

class MarketRegime:
    """Separate class to handle regime logic and transitions"""
    BULL = "bull"
    BEAR = "bear"
    CORRECTION = "correction"
    RECOVERY = "recovery"
    
    def __init__(self):
        # Regime transition thresholds
        self.thresholds = {
            'bear_market': -0.20,      # -20% from peak defines bear market
            'correction': -0.10,        # -10% from peak defines correction
            'recovery_threshold': 0.15,  # +15% from trough signals recovery
            'bull_confirmation': 0.05   # +5% from support confirms new bull
        }
        
        # Lookback windows for regime detection
        self.windows = {
            'peak_window': 60,         # Days to look back for peak
            'support_window': 30,      # Days to confirm support level
            'trend_window': 20         # Days for trend calculation
        }

    def detect_regime(self, prices, index):
        """
        Detect current regime based on price action rules
        """
        if index < self.windows['peak_window']:
            return self.BULL
            
        current_price = prices[index]
        
        # Find recent peak and trough
        lookback = min(index, self.windows['peak_window'])
        price_window = prices[index - lookback:index + 1]
        peak = np.max(price_window)
        trough = np.min(price_window)
        
        # Calculate drawdown from peak
        drawdown = (current_price / peak) - 1
        
        # Calculate rally from trough
        rally = (current_price / trough) - 1
        
        # Calculate recent trend
        trend_window = min(self.windows['trend_window'], index)
        recent_trend = (current_price / prices[index - trend_window]) - 1
        
        # Regime detection logic
        if drawdown <= self.thresholds['bear_market']:
            # In bear market
            if rally >= self.thresholds['recovery_threshold']:
                return self.RECOVERY
            return self.BEAR
            
        elif drawdown <= self.thresholds['correction']:
            # In correction
            if recent_trend > self.thresholds['bull_confirmation']:
                return self.BULL
            return self.CORRECTION
            
        else:
            # Check if recovering
            if self._was_recent_regime(prices, index, self.BEAR, lookback=30):
                if rally < self.thresholds['recovery_threshold']:
                    return self.RECOVERY
            
            return self.BULL
    
    def _was_recent_regime(self, prices, current_idx, regime, lookback=30):
        """Check if a regime occurred recently in the lookback window"""
        if current_idx < lookback:
            return False
        
        start_idx = max(0, current_idx - lookback)
        price_window = prices[start_idx:current_idx + 1]
        
        if regime == self.BEAR:
            max_drawdown = (np.min(price_window) / np.max(price_window)) - 1
            return max_drawdown <= self.thresholds['bear_market']
            
        return False

class SyntheticMarketGenerator:
    def __init__(self,
                 global_seed=None,
                 trading_days_per_year=252,
                 years=10,
                 default_bull_drift=0.15,     # 15% annual drift in bull markets
                 default_bear_drift=-0.35,    # -35% annual drift in bear markets
                 default_upward_bias=0.02,    # 2% general upward bias
                 default_bull_vol=0.15,       # 15% annual vol in bull markets
                 default_bear_vol=0.35):      # 35% annual vol in bear markets
        
        if global_seed is not None:
            np.random.seed(global_seed)

        self.trading_days_per_year = trading_days_per_year
        self.years = years
        self.total_days = trading_days_per_year * years
        
        # Base parameters
        self.default_bull_drift = default_bull_drift
        self.default_bear_drift = default_bear_drift
        self.default_upward_bias = default_upward_bias
        self.default_bull_vol = default_bull_vol
        self.default_bear_vol = default_bear_vol
        
        # Initialize regime manager
        self.regime_manager = MarketRegime()
        
        # Market dynamics parameters
        self.params = {
            'vol_mean_reversion': 0.90,      # Volatility mean reversion speed
            'vol_of_vol': 0.08,              # Volatility of volatility
            'base_vol': 0.10,                # Minimum annualized volatility
            'max_vol': 0.50,                 # Maximum annualized volatility
            'momentum_decay': 0.98,          # Decay factor for price momentum
            'momentum_impact': 0.10,         # How much momentum affects drift
            'earnings_jump_prob': 0.015,     # Probability of earnings jumps
            'earnings_jump_range': (-0.15, 0.15),  # Range for earnings jumps
            'flash_crash_prob': 0.0005,      # Probability of flash crashes
            'flash_crash_range': (-0.15, -0.07),  # Range for flash crashes
        }

    def generate_stock_data(self, ticker="STK", initial_price=None):
        """Generate synthetic stock data with realistic regime behavior"""
        
        # Initialize price arrays
        dates = self._generate_dates_with_offset()
        N = len(dates)
        
        close_prices = np.zeros(N)
        open_prices = np.zeros(N)
        high_prices = np.zeros(N)
        low_prices = np.zeros(N)
        volumes = np.zeros(N)
        daily_vols = np.zeros(N)
        regimes = np.empty(N, dtype=object)
        log_returns = np.zeros(N)
        
        # Set initial price
        if initial_price is None:
            initial_price = np.random.uniform(50, 150)
        
        close_prices[0] = initial_price
        daily_vols[0] = self.default_bull_vol / np.sqrt(self.trading_days_per_year)
        regimes[0] = self.regime_manager.BULL
        
        # Initialize first day OHLC
        open_prices[0], high_prices[0], low_prices[0] = self._generate_first_day_ohlc(initial_price, daily_vols[0])
        volumes[0] = self._generate_volume(0, daily_vols[0], regimes[0])
        
        # Initialize state variables
        momentum = 0.0
        volatility_regime = 1.0  # Multiplier for base volatility
        
        # Generate price series
        for i in range(1, N):
            # Detect current regime based on price action
            current_regime = self.regime_manager.detect_regime(close_prices, i-1)
            regimes[i] = current_regime
            
            # Get base drift and volatility for current regime
            drift, vol = self._get_regime_parameters(current_regime)
            
            # Apply momentum effect
            drift += momentum * self.params['momentum_impact']
            
            # Calculate daily parameters
            daily_drift = np.log(1 + drift) / self.trading_days_per_year
            daily_drift += self.default_upward_bias / self.trading_days_per_year
            
            # Update volatility
            target_vol = vol * volatility_regime
            daily_vols[i] = self._update_volatility(daily_vols[i-1], target_vol)
            
            # Generate log return
            log_returns[i] = self._generate_return(daily_drift, daily_vols[i])
            
            # Apply special events
            if current_regime not in [self.regime_manager.BEAR, self.regime_manager.CORRECTION]:
                log_returns[i] = self._apply_special_events(log_returns[i])
            
            # Update price
            close_prices[i] = close_prices[i-1] * np.exp(log_returns[i])
            
            # Generate OHLC and volume
            open_prices[i], high_prices[i], low_prices[i] = self._generate_ohlc(
                close_prices[i-1], close_prices[i], daily_vols[i], current_regime
            )
            volumes[i] = self._generate_volume(log_returns[i], daily_vols[i], current_regime)
            
            # Update state variables
            momentum = momentum * self.params['momentum_decay'] + log_returns[i] * (1 - self.params['momentum_decay'])
            volatility_regime = self._update_volatility_regime(volatility_regime, log_returns[i], current_regime)

        # Create DataFrame
        df = pd.DataFrame({
            'Date': dates,
            'Open': open_prices,
            'High': high_prices,
            'Low': low_prices,
            'Close': close_prices,
            'Volume': volumes.astype(int),
            'Regime': regimes,
            'Volatility': daily_vols,
            'LogReturn': log_returns
        })
        df.set_index('Date', inplace=True)
        
        return df

    def _generate_dates_with_offset(self):
        """Generate dates with random start offset"""
        offset = np.random.randint(0, 61)
        start_date = datetime.datetime(2010,1,1) + datetime.timedelta(days=offset)
        
        dates = []
        current = start_date
        while len(dates) < self.total_days:
            if current.weekday() < 5:  # Monday to Friday
                dates.append(current)
            current += datetime.timedelta(days=1)
        return dates

    def _get_regime_parameters(self, regime):
        """Get drift and volatility parameters for given regime"""
        if regime == self.regime_manager.BULL:
            return self.default_bull_drift, self.default_bull_vol
        elif regime == self.regime_manager.BEAR:
            return self.default_bear_drift, self.default_bear_vol
        elif regime == self.regime_manager.CORRECTION:
            return self.default_bear_drift * 0.5, (self.default_bull_vol + self.default_bear_vol) * 0.5
        elif regime == self.regime_manager.RECOVERY:
            return self.default_bull_drift * 1.5, self.default_bear_vol * 0.7
        else:
            return self.default_bull_drift, self.default_bull_vol

    def _update_volatility(self, current_vol, target_vol):
        """Update daily volatility with mean reversion"""
        daily_target = target_vol / np.sqrt(self.trading_days_per_year)
        daily_vol = (
            self.params['vol_mean_reversion'] * daily_target +
            (1 - self.params['vol_mean_reversion']) * current_vol +
            np.random.normal(0, self.params['vol_of_vol'] / np.sqrt(self.trading_days_per_year))
        )
        
        # Apply bounds
        min_daily_vol = self.params['base_vol'] / np.sqrt(self.trading_days_per_year)
        max_daily_vol = self.params['max_vol'] / np.sqrt(self.trading_days_per_year)
        return np.clip(daily_vol, min_daily_vol, max_daily_vol)

    def _generate_return(self, drift, vol):
        """Generate daily return with fat tails"""
        # Use Student's t distribution for fat tails
        shock = student_t.rvs(df=5)  # df=5 gives fatter tails than normal
        shock /= np.sqrt(5/3)  # Scale to unit variance
        return drift + vol * shock

    def _apply_special_events(self, daily_return):
        """Apply special events like earnings jumps or flash crashes"""
        if np.random.random() < self.params['flash_crash_prob']:
            return np.random.uniform(*self.params['flash_crash_range'])
            
        if np.random.random() < self.params['earnings_jump_prob']:
            jump = np.random.uniform(*self.params['earnings_jump_range'])
            return daily_return + jump
            
        return daily_return

    def _generate_first_day_ohlc(self, initial_price, daily_vol):
        """Generate OHLC for the first day"""
        open_dev = np.random.normal(0, daily_vol)
        open_price = initial_price * (1 + open_dev)
        
        if open_dev > 0:
            high_price = max(open_price, initial_price) * (1 + abs(np.random.normal(0, daily_vol)))
            low_price = min(open_price, initial_price) * (1 - abs(np.random.normal(0, daily_vol * 0.5)))
        else:
            high_price = max(open_price, initial_price) * (1 + abs(np.random.normal(0, daily_vol * 0.5)))
            low_price = min(open_price, initial_price) * (1 - abs(np.random.normal(0, daily_vol)))
            
        # Ensure OHLC relationships
        high_price = max(high_price, open_price, initial_price)
        low_price = min(low_price, open_price, initial_price)
        
        return open_price, high_price, low_price

    def _generate_ohlc(self, prev_close, curr_close, daily_vol, regime):
        """Generate intraday OHLC prices"""
        price_change = curr_close - prev_close
        
        # Generate open with overnight gap
        gap_vol = daily_vol * 1.5 if regime in [self.regime_manager.BEAR, self.regime_manager.CORRECTION] else daily_vol
        open_price = prev_close + np.random.normal(0, gap_vol * prev_close)
        
        # Base range on volatility and regime
        range_factor = 2.0 if regime in [self.regime_manager.BEAR, self.regime_manager.CORRECTION] else 1.5
        price_range = daily_vol * range_factor * prev_close
        
        # Generate high/low based on price direction
        if curr_close > prev_close:
            high_price = max(open_price, curr_close) + abs(np.random.normal(0, price_range * 0.5))
            low_price = min(open_price, curr_close) - abs(np.random.normal(0, price_range * 0.2))
        else:
            high_price = max(open_price, curr_close) + abs(np.random.normal(0, price_range * 0.2))
            low_price = min(open_price, curr_close) - abs(np.random.normal(0, price_range * 0.5))
            
        # Ensure OHLC relationships
        high_price = max(high_price, open_price, curr_close)
        low_price = min(low_price, open_price, curr_close)
        
        return open_price, high_price, low_price

    def _generate_volume(self, log_return, daily_vol, regime):
        """Generate trading volume based on price action and regime"""
        base_volume = 1_000_000
        
        # Volume factors
        vol_factor = (daily_vol * np.sqrt(self.trading_days_per_year) / self.default_bull_vol)
        return_factor = 1 + 2.0 * abs(log_return) / daily_vol if daily_vol > 0 else 1
        regime_factor = {
            self.regime_manager.BULL: 1.0,
            self.regime_manager.BEAR: 2.0,
            self.regime_manager.CORRECTION: 1.5,
            self.regime_manager.RECOVERY: 1.3
        }.get(regime, 1.0)
        
        # Random variation
        noise = np.random.lognormal(0, 0.5)
        
        return int(base_volume * vol_factor * return_factor * regime_factor * noise)

    def _update_volatility_regime(self, current_regime, log_return, market_regime):
        """Update volatility regime based on returns and market conditions"""
        target = 1.0
        
        # Increase vol regime in bear markets and corrections
        if market_regime in [self.regime_manager.BEAR, self.regime_manager.CORRECTION]:
            target = 1.5
        
        # Adjust based on return magnitude
        if abs(log_return) > 3 * self.default_bull_vol / np.sqrt(self.trading_days_per_year):
            target *= 1.2
        
        # Mean revert
        return 0.95 * current_regime + 0.05 * target

    def plot_stock(self, df, ticker, save_path=None):
        """Plot stock price with regime backgrounds"""
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), height_ratios=[3, 1])
        
        # Plot price
        ax1.plot(df.index, df['Close'], 'k-', linewidth=1, label='Close')
        
        # Color backgrounds for regimes
        regime_colors = {
            self.regime_manager.BULL: 'lightgreen',
            self.regime_manager.BEAR: 'lightcoral',
            self.regime_manager.CORRECTION: 'lightyellow',
            self.regime_manager.RECOVERY: 'lightblue'
        }
        
        ymin, ymax = df['Close'].min(), df['Close'].max()
        padding = (ymax - ymin) * 0.1
        ax1.set_ylim(ymin - padding, ymax + padding)
        
        # Plot regime backgrounds
        for regime, color in regime_colors.items():
            mask = (df['Regime'] == regime)
            if mask.any():
                ax1.fill_between(
                    df.index, ymin - padding, ymax + padding,
                    where=mask, color=color, alpha=0.3, label=regime
                )
        
        # Plot volume
        ax2.bar(df.index, df['Volume'], color='gray', alpha=0.5)
        ax2.set_ylabel('Volume')
        
        # Formatting
        ax1.set_title(f'{ticker} Synthetic Price', fontsize=14)
        ax1.set_ylabel('Price')
        ax1.grid(True, alpha=0.3)
        ax1.legend(loc='upper left')
        
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    def generate_portfolio(self, num_stocks=5, output_dir="synthetic_portfolio"):
        """Generate multiple stocks with output files"""
        os.makedirs(output_dir, exist_ok=True)
        portfolio = {}
        
        for i in range(num_stocks):
            # Generate random ticker
            ticker = ''.join(np.random.choice(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 4))
            
            # Generate data
            df = self.generate_stock_data(ticker=ticker)
            portfolio[ticker] = df
            
            # Save data
            df.to_csv(os.path.join(output_dir, f"{ticker}.csv"))
            self.plot_stock(df, ticker, save_path=os.path.join(output_dir, f"{ticker}.png"))
            
            # Save metadata
            metadata = {
                'ticker': ticker,
                'start_date': df.index[0].strftime('%Y-%m-%d'),
                'end_date': df.index[-1].strftime('%Y-%m-%d'),
                'initial_price': df['Close'][0],
                'final_price': df['Close'][-1],
                'total_return': (df['Close'][-1] / df['Close'][0] - 1) * 100,
                'max_drawdown': self._calculate_max_drawdown(df['Close']),
                'annualized_volatility': np.std(df['LogReturn']) * np.sqrt(self.trading_days_per_year)
            }
            
            with open(os.path.join(output_dir, f"{ticker}_metadata.json"), 'w') as f:
                json.dump(metadata, f, indent=2)
        
        return portfolio

    def _calculate_max_drawdown(self, prices):
        """Calculate maximum drawdown percentage"""
        peak = prices[0]
        max_drawdown = 0
        
        for price in prices:
            if price > peak:
                peak = price
            drawdown = (peak - price) / peak
            max_drawdown = max(max_drawdown, drawdown)
        
        return max_drawdown * 100

def main():
    """Example usage"""
    generator = SyntheticMarketGenerator(global_seed=42)
    portfolio = generator.generate_portfolio(num_stocks=5, output_dir="snplike_ticker_portfolio")
    
if __name__ == "__main__":
    main()